In [15]:
# In Fixed window, every window is fixed and we number each window.
# So, we have seen the issue of requests coming at edge of previous window.
# In sliding window, the window is sliding. We count the window starting from
# current_time - window_size (in seconds) to current_time. And we maintain a queue
# of requests. If we see that there is a request in queue having ts of < current_time - window_size,
# then we pop it out. So, we need to log every request actually.

import time as TIME
from collections import deque

class SlidingWindowLog():
    def __init__(self, windowSize, maxReq):
        # So, we need windowSize in seconds and maxReq
        # allowed in a window.
        self.windowSize = windowSize
        self.maxReq = maxReq
        
        # There is no need to save the current time,
        # we will pop out reqs from the queue in runtime
        # depending on the time.
        
        self.bucket = deque()
        
    def handle_request(self, req):
        # Surely we need to take the request in real use cases.
        # Here request timestamp will be calculated here only.
        
        requestTime = TIME.time()
        lastAllowedTimeInQueue = requestTime - self.windowSize
        
        # Now, check the queue and remove the ones having req time < lastAllowedTimeInQueue
        # All are in seconds so comparison is easier.
        
        while(len(self.bucket) > 0 and self.bucket[0] < lastAllowedTimeInQueue):
            self.bucket.popleft()
            
        # Now that old requests are removed. We need to add this req. But we must be careful
        # and check whether it is crossing the maxReq in thsi window.
        
        if(len(self.bucket) >= self.maxReq):
            print("Request Dropped...")
            return False
        
        # Otherwise surely we can serve this request.
        self.bucket.append(requestTime) # We can save (req, requestTime) pair also IRL.
        print("Request Served...")
        return True

In [16]:
limiter = SlidingWindowLog(windowSize = 10, maxReq = 5) # So, in 10 seconds sliding window, we allow 5 req at max.

for req in range(10):
    print(f"This is request {req+1}")
    limiter.handle_request(req)
    time.sleep(0.4)
    
time.sleep(7)
# So, we wait for 7 more seconds to make our window slide.
print(f"This is a new request")
limiter.handle_request(100)

This is request 1
Request Served...
This is request 2
Request Served...
This is request 3
Request Served...
This is request 4
Request Served...
This is request 5
Request Served...
This is request 6
Request Dropped...
This is request 7
Request Dropped...
This is request 8
Request Dropped...
This is request 9
Request Dropped...
This is request 10
Request Dropped...
This is a new request
Request Served...


True